# ACCESS-OM2 Salinity

In [1]:
import intake

import matplotlib.pyplot as plt
import xarray as xr
import cf_xarray as cfxr
import numpy as np

import cmocean as cm
import cartopy.crs as ccrs

import cartopy.feature as cft
import matplotlib.path as mpath

In [2]:
acccontrol = intake.open_esm_datastore('/home/272/kc5856/access_control_new.json', columns_with_iterables=['variable'])
accmelt = intake.open_esm_datastore('/home/272/kc5856/access_meltwater_ssp126.json', columns_with_iterables=['variable'])

In [3]:
#Starting a dask client
from os import environ
environ["PYTHONWARNINGS"] = "ignore"

from dask.distributed import Client
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 126.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38169,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 126.00 GiB
Comm: tcp://127.0.0.1:36383,Total threads: 1
Dashboard: /proxy/42045/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:44869,


In [4]:
#Time variant variables #ASK HOW TO CONDENSE
control_SO = acccontrol.search(variable="salt", frequency="1mon").to_dask()
melt_SO = accmelt.search(variable="salt", frequency="1mon").to_dask()

2025-05-08 13:40:52,854 - distributed.worker - ERROR - Compute Failed
Key:       _open_dataset-f93a54c6-7fc3-457f-83fe-efef1cdff2f4
State:     executing
Task:  <Task '_open_dataset-f93a54c6-7fc3-457f-83fe-efef1cdff2f4' apply(...)>
Exception: "PermissionError(13, 'Permission denied')"
Traceback: '  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.04/lib/python3.11/site-packages/dask/utils.py", line 77, in apply\n    return func(*args, **kwargs)\n           ^^^^^^^^^^^^^^^^^^^^^\n  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.04/lib/python3.11/site-packages/intake_esm/source.py", line 71, in _open_dataset\n    url = fsspec.open_local(urlpath, **storage_options)\n          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.04/lib/python3.11/site-packages/fsspec/core.py", line 533, in open_local\n    with of as files:\n  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.04/lib/python3.11/site-packages

ESMDataSourceError: Failed to load dataset with key='ocean.1mon'
                 You can use `cat['ocean.1mon'].df` to inspect the assets/files for this key.
                 

In [ ]:
#Area weighting
data_fixed = acccontrol.search(variable=["area_t","ht"],path=".*output021.*").to_dask()
area = data_fixed.area_t
vol = acccontrol.search(variable=["volcello"],path=".*output021.*"]).to_dask()
thick = vol / area
thick = thick.load()

In [ ]:
def shelf_mask_isobath(var, model_dict):

    paths = {
             "mom5": "/g/data/ik11/grids/Antarctic_slope_contour_1000m.npz",
             "mom6": "/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_01deg.nc",
            "mom6_005": "/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_005deg.nc"
             }

    var = var.cf.sel({'latitude': slice(-90, -59)})

    if paths[model_dict][-3:] == '.nc':
        shelf_mask = xr.open_dataset(paths[model_dict])['contour_masked_above']
    else:
        contour_file = np.load(paths[model_dict])
        shelf_mask = xr.DataArray(contour_file['contour_masked_above'],
                                  coords = var.coords, 
                                  dims = var.dims,
                                  name = 'contour_masked_above')
    
    shelf_mask = xr.where(shelf_mask == 0, 1, 0)
    masked_var = var * shelf_mask
    
    return masked_var, shelf_mask

In [ ]:
depth = data_fixed.ht
land_mask = (depth*0).fillna(1)
depth_shelf, shelf_mask = shelf_mask_isobath(depth, 'mom5')

In [ ]:
cont01_200_map = control01_salt.so.sel(time=slice('1918-01-01','1919-12-31')).sel(z_l=slice(0,220)).weighted(thick01['z_l'].fillna(0)).mean(dim='z_l').sel(yh=slice(None,-59)).mean(dim='time')
pert01_200_map = trial01_salt.so.sel(time=slice('1918-01-01','1919-12-31')).sel(z_l=slice(0,220)).weighted(thick01['z_l'].fillna(0)).mean(dim='z_l').sel(yh=slice(None,-59)).mean(dim='time')